# Development Notebook for Mimir
* POC looking at querying firebase and implementing a radar mode.
* Looking to successfully query Firebase via the API.
* Think about how the data coming in should be formatted.
* How should the results be displayed?

## Radar Mode Flow
1. When performing a Radar Mode request - let's say for instance a user is on the app and say I request a Radar Mode report for sensor 1.
2. This now fires off a GET request to our API.
3. From the request we should receive a UNIQUE DEVICE ID.
4. Can now query the sensor data table in order to get the latest sensor reading.
5. Request should also include a plant ID - used to look up in the plant data table and retrieve data
6. Peform the analysis and return data to the user

* _Data is currently in the wrong form_ :

userInfo
deviceId
"P4006"
ipAddress
"62.203.6.17"
macAddress
"24:6F:28:B1:DC:D8"
userId
"Desk Top"
userName
"Lloyd"
version
"P2004.11C"

* We do have a deviceId - not sure if this is unique - so will use this for now.

## 1. Obtain Correct Sensor Data

In [1]:
# Example - get all readings for a certain user/ device
import requests
import json
from pprint import pprint

# Receive a Radar Mode Request from a User
# They will have to select a device and a plant that they want a reading for
deviceId = "P4006" # should be unique
userId = "xyz" # would ordinarily get a user ID as well

# They would say which species they are performing the analysis for
plant_species = "aloe vera" # would normally be a plant ID

# Build up a URL for request
base_url = "https://firestore.googleapis.com/v1/projects/mimirhome-app/databases/(default)/documents/"

# Get the relevant sensor data
sensor_data = requests.get(base_url + "sensorData/").text

# Obtain in json format
sensor_json = json.loads(sensor_data)

# Obtain all entries that match the deviceId
device_data = [s for s in sensor_json['documents'] if s['fields']['userInfo']['mapValue']['fields']['deviceId']['stringValue'] == deviceId]

In [2]:
device_data

[{'name': 'projects/mimirhome-app/databases/(default)/documents/sensorData/00ebE6UhN4wVIJeMU8Az',
  'fields': {'status': {'mapValue': {'fields': {'WiFI_Status': {'integerValue': '1'},
      'MicroSD_Status': {'integerValue': '1'},
      'WiFi_Signal': {'integerValue': '-59'},
      'Battery_Percent': {'integerValue': '-6'},
      'Date': {'stringValue': '2020-05-31'},
      'Time': {'stringValue': '15:30:44'},
      'Sensors_Status': {'integerValue': '1'},
      'Battery_Status': {'integerValue': '0'},
      'Server_Status': {'integerValue': '1'}}}},
   'userInfo': {'mapValue': {'fields': {'userId': {'stringValue': 'Desk Top'},
      'version': {'stringValue': 'P2004.11C'},
      'ipAddress': {'stringValue': '62.203.6.17'},
      'macAddress': {'stringValue': '24:6F:28:B1:DC:D8'},
      'deviceId': {'stringValue': 'P4006'},
      'userName': {'stringValue': 'Lloyd'}}}},
   'data': {'mapValue': {'fields': {'VOC(CCS811)': {'integerValue': '194'},
      'Humidity(SHT31_L)': {'doubleValue'

In [3]:
# Now have device data in a list - find the most recent 
from datetime import datetime as dt

ts = [dt.strptime(d['createTime'][:19], "%Y-%m-%dT%H:%M:%S") for d in device_data]  # 2020-05-27T08:30:46.607679Z

# Currently just reading one datapoint although I think it would be good to use an
# average depending on the frequency of the reads.

read_data = device_data[ts.index(min(ts))]

In [4]:
read_data

{'name': 'projects/mimirhome-app/databases/(default)/documents/sensorData/04RJS7GERbCg6YzZqF2Q',
 'fields': {'timestamp': {'integerValue': '1590568246459'},
  'status': {'mapValue': {'fields': {'Sensors_Status': {'integerValue': '1'},
     'MicroSD_Status': {'integerValue': '1'},
     'WiFI_Status': {'integerValue': '1'},
     'Date': {'stringValue': '2020-05-27'},
     'Battery_Percent': {'integerValue': '92'},
     'Battery_Status': {'integerValue': '2'},
     'Server_Status': {'integerValue': '1'},
     'WiFi_Signal': {'integerValue': '-63'},
     'Time': {'stringValue': '10:30:42'}}}},
  'data': {'mapValue': {'fields': {'Luminance(VEML6030)': {'integerValue': '693'},
     'Pressure(BMP280)': {'doubleValue': 982.4601},
     'Altitude(BMP280)': {'doubleValue': 982.4601},
     'UVB(VEML6075)': {'doubleValue': 25.9},
     'eCO2(CCS811)': {'integerValue': '1902'},
     'UVA(VEML6075)': {'doubleValue': 43.49},
     'Temperature(SHT31_L)': {'doubleValue': 24.89},
     'Temperature(SHT31_H

## 2. Obtain Plant Data

In [7]:
# Get the relevant sensor data
plant_data = requests.get(base_url + "plants/" + pname).text
plant_json = json.loads(plant_data)
plant_json

NameError: name 'name' is not defined

## 3. Perform Analysis

In [6]:
import numpy
import skfuzzy as fuzz

In [7]:
ideal_dict = plant_json['fields']['parameter']['mapValue']['fields']
actual_dict = read_data['fields']['data']['mapValue']['fields']
print('Ideal Ranges: \n')
pprint(ideal_dict)
print('Actual Readings : \n')
pprint(actual_dict)

Ideal Ranges: 

{'max_env_humid': {'integerValue': '80'},
 'max_light_lux': {'integerValue': '70000'},
 'max_light_mmol': {'integerValue': '6400'},
 'max_soil_ec': {'integerValue': '1000'},
 'max_soil_moist': {'integerValue': '50'},
 'max_temp': {'integerValue': '35'},
 'min_env_humid': {'integerValue': '15'},
 'min_light_lux': {'integerValue': '3000'},
 'min_light_mmol': {'integerValue': '3200'},
 'min_soil_ec': {'integerValue': '300'},
 'min_soil_moist': {'integerValue': '7'},
 'min_temp': {'integerValue': '8'}}
Actual Readings : 

{'Altitude(BMP280)': {'doubleValue': 982.4601},
 'Bearing(Compass)': {'integerValue': '90'},
 'Humidity(SHT31_H)': {'doubleValue': 35.48},
 'Humidity(SHT31_L)': {'doubleValue': 35.84},
 'Luminance(VEML6030)': {'integerValue': '693'},
 'Pressure(BMP280)': {'doubleValue': 982.4601},
 'Temperature(SHT31_H)': {'doubleValue': 24.8},
 'Temperature(SHT31_L)': {'doubleValue': 24.89},
 'Temperature(bmp280)': {'doubleValue': 26.23},
 'UVA(VEML6075)': {'doubleValue':

In [55]:
class EnvironmentalAnalysis():
    
    # Imports
    import numpy as np
    
    def __init__(self, species):
        
        self.species = species # plant species
    
    @staticmethod
    def rangeFunc(min, max, val):
        """
        Class method to look at a value compared to its ideal range.

        Args:
            min (float): minimum value in the range.
            max (float): maximum value in the range.
            val (float): recorded value

        Returns:
            position (float): relative position between the extremes.

        """
        # Get the relative postion of the value between the extremes
        # If it is at the max then it will be 1
        # If it is at the min then it will be 0
        postion = (val - min)/(max - min)
        return postion
             
        
    def radarMode(self, ideal, actual):
        """
        Class method to perform Radar Analysis mode

        Args:
            ideal (dict): Dictionary containing all the ideal conditions for the given species.
            actual (dict): Sensor Readings for the actual environmental conditons.

        Returns:   
            result (dict): Dictionary containing the results of the radar analysis

        """
    
        # Create the extreme dictionary (sounds extreme)
        extreme_dict = {}
        extreme_dict['humidity'] = (int(ideal['min_env_humid']['integerValue']), int(ideal['max_env_humid']['integerValue']))
        extreme_dict['temperature'] = (int(ideal['min_temp']['integerValue']), int(ideal['max_temp']['integerValue']))
        extreme_dict['light'] = (int(ideal['min_light_lux']['integerValue']), int(ideal['max_light_lux']['integerValue']))
        
        print(extreme_dict)

        # Create dictionary containing actual readings
        reading_dict = {}
        reading_dict['humidity'] = self.np.mean([actual['Humidity(SHT31_H)']['doubleValue'], actual['Humidity(SHT31_L)']['doubleValue']])
        reading_dict['temperature']  = self.np.mean([actual['Temperature(SHT31_H)']['doubleValue'], actual['Temperature(SHT31_L)']['doubleValue']])
        reading_dict['light'] = int(actual['Luminance(VEML6030)']['integerValue'])
        
        print(reading_dict)

        # Get Results 
        result_dict = {}
        for condition in ['humidity', 'temperature', 'light']:
            result_dict[condition] = self.rangeFunc(extreme_dict[condition][0], extreme_dict[condition][1], reading_dict[condition])

        return result_dict
    
    def natlangFeedback(self, result_dict):
        """
        Class method to provide some natural language feedback given results
        
        Args:
            result_dict (dict): Dictionary of results from an analysis mode.
            
        """
       
        for k, v in result_dict.items():
            if v <= 0:
                print(f"The {k} is dangerously low for {self.species}.")
            elif v > 0 and v <= 0.25:
                print(f"The {k} is very low for {self.species}.")
            elif v > 0.25 and v <= 0.75:
                print(f"The {k} is a good level for {self.species}.")
            elif v > 0.75 and v <= 1:
                print(f"The {k} is a very high for {self.species}.")
            else:
                print(f"The {k} is a dangerously high for {self.species}.")                                         

In [56]:
# Initialise a class
analysis = EnvironmentalAnalysis(plant_species)
results = analysis.radarMode(ideal_dict, actual_dict)
print(results)

{'humidity': (15, 80), 'temperature': (8, 35), 'light': (3000, 70000)}
{'humidity': 35.66, 'temperature': 24.845, 'light': 693}
{'humidity': 0.3178461538461538, 'temperature': 0.6238888888888888, 'light': -0.03443283582089552}


In [57]:
analysis.natlangFeedback(results)

The humidity is a good level for aloe vera.
The temperature is a good level for aloe vera.
The light is dangerously low for aloe vera.
